In [1]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
from statsmodels.datasets import grunfeld
from linearmodels.panel  import PanelOLS
import pandas as pd
import build_data_functions as bdf
import plot_data_functions as pdf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import model_functions as mf
import plot_model_functions as pmf
import panelOLS_models 
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import pycountry
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import normalize

from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from scipy.stats import normaltest

In [2]:
years = list(range(2005, 2017))

In [3]:
y = pd.read_table("/home/sara/Documents/Immigration/Shared_models/Data/resident_foreigners_norm.csv", sep = "\t", index_col=0)
y = y.groupby(["Province", "Country", "Year"], as_index=False)["Value"].sum()
y = bdf.pivot(y, "Country", "Value")

resident_foreigners_norm = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/resident_foreigners_norm.csv")

#unesco_demo = pd.read_table("/home/sara/Documents/Immigration/Shared_models/Data/countries_info.csv", sep = "\t", index_col=[0,1])

## Italian Stock Prediction

In [4]:
# To get the Italy info we need to sum the 5 Italian zones
zones = list(pd.read_table("/home/sara/Documents/Immigration/Shared_models/Data/x_zones.csv", sep = "\t", index_col=["Province", "Year"]).index.levels[0])

for t in y.index.levels[1]:
    y.loc[("Italia", t), :] = y.loc[(zones, t), :].sum()

In [5]:
temp = pd.read_table("/home/sara/Documents/Immigration/Shared_models/Data/x_zones.csv", sep = "\t", index_col=["Province", "Year"])
xs = pd.DataFrame(columns=["Province", "Year"]+temp.columns.tolist())
xs = xs.set_index(["Province","Year"])

for t in temp.index.levels[1]:
    xs.loc[("Italia", t), :] = temp.loc[(zones, t), :].sum()

In [6]:
palette = ['blue', 'darkgreen', 'yellowgreen', 'orange', 'lightcoral', 'red', 'paleturquoise', 'deepskyblue', 'mediumpurple', 'fuchsia']

In [9]:
countries_list = ["Romania", "Morocco", "Albania", "Tunisia", "Egypt", "Ecuador", "Peru", "China", "Philippines"]
countries_list_iso3 = [pycountry.countries.get(name=country).alpha_3 for country in countries_list]
target = "Italia"
# number of features to select
ks = [3, 5, 7, 10, 15]

In [10]:
y_hat = mf.compute_regression_model(y, xs, years, countries_list, target, ks)

ROU
['y_prev_1', 'native population - Total', 'Average age of mothers at birth', 'Disposable Income', 'Free activities in voluntary associations']
MAR
['y_prev_1', 'Disposable Income', 'y_prev_2', 'native population - Total', 'Free activities in voluntary associations', 'Average age of mothers at birth', 'Average age of fathers at birth', 'political_info - Some times in a week', 'Average monthly expenditure for housing', 'unemployment - Total', 'internal_migration - Foreign country', 'political_info - A time per week', 'internal_migration - Italy', 'reach_difficulty - Pharmacy', 'Furniture, articles and services for the house']
ALB
['y_prev_1', 'Disposable Income', 'y_prev_2', 'native population - Total', 'Average age of mothers at birth', 'Free activities in voluntary associations', 'Average age of fathers at birth', 'political_info - Some times in a week', 'Average monthly expenditure for housing', 'internal_migration - Italy', 'internal_migration - Foreign country', 'Communications'

In [11]:
pdf.relation_plot_time_variant_intern_function(y_hat, countries_list_iso3, years, ["Predicted"], resident_foreigners_norm.groupby(["Country", "Year"]), plt.figure(1, figsize=(15,14)), 331, 45, palette, None, "Immigrant Stock Real VS Predicted", True, "Plots/regression_model_italy", False)

In [ ]:
## Zones Prediction - au